***參考教學網站：[https://www.tensorflow.org/hub/tutorials/object_detection](https://www.tensorflow.org/hub/tutorials/object_detection)***

In [ ]:
!pip install tensorflow
!pip install matplotlib
!pip install tf_slim
!pip install pycocotools
!pip install pillow
!pip install requests
!pip install lxml

In [ ]:
 !pip install --upgrade tensorflow_hub

In [ ]:
import platform
print(platform.system(),"\n",platform.uname())

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tempfile
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
from IPython.display import display as ds
import time
import pathlib
import requests
import shutil
from lxml import etree
import os

In [ ]:
print(tf.__version__)

In [ ]:
path =  os.getcwd() 

if(not os.path.exists(path+r"\images")):
   os.mkdir("images")

path=path.replace("\\", "/")

path=path+"/images/"

print( path )

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [ ]:
def display_image(image):
  #fig = plt.figure(figsize=(20, 30))
  #plt.grid(False)
  #plt.imshow(image)
  ds(image)

def load_image(url, new_width, new_height,display=False):
  if display:
    display_image(Image.open(url))
  return url

In [ ]:
def download_and_resize_image(url, new_width, new_height,display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  #response = urlopen(url)
  #image_data = response.read()
  #image_data = BytesIO(image_data)
  image_data = url
  new_width1=int(new_width/3) 
  new_height1=int(new_height/3) 
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image,(new_width1,new_height1), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=100)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

In [ ]:
my_imgs_now=[]
dirPath = os.walk(path)
img_dir_path=list(dirPath)[0][2]
for i in img_dir_path:
    if(i.split('.')[-1]=="jpg"):
        my_imgs_now.append(i)
#list(enumerate(dirPath))[0]

def get_correct_url(base_url,part_url):
  if(part_url.split('/')[0].find('http')==-1):
    if(part_url.split('/')[0]==''):
      if(part_url.split('/')[0]==''):
        if(part_url.split('/')[1]==''):
          if(part_url.split('/')[2]==base_url.split('/')[2]):
            return "http:"+part_url
          else:
            pass  
        else:
          if(part_url.split('/')[1]==base_url.split('/')[2]):
            return "http:/"+part_url
          else:
            return "http://"+base_url.split("/")[2]+part_url 
      else:  
        return "http://"+base_url.split("/")[2]+"/"+part_url 
    else:
      return "http://"+base_url.split("/")[2]+"/"+part_url       

  else:
    return part_url



def png_to_jpg(myImgName):
  im = Image.open( path+mypath )
  rgb_im = im.convert('RGB')
  rgb_im.save(path+myImgName+".jpg")

def my_save_img():
  global my_imgs_now
  def my_save_url_img(my_img_url): 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
    r = requests.get(my_img_url,headers=headers, stream=True)
    if r.status_code == 200:
        with open(str((path+my_img_url.split('/')[-1].split('?')[0])), 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

  url_01=input("請輸入圖片網址：")
  url_01_type=url_01.split('/')[-1].split('.')[-1].split('?')[0]
  
  if(url_01_type=='jpg'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])
  
  elif(url_01_type=='png'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0]+".jpg")
    png_to_jpg(url_01.split('/')[-1].split('?')[0])

  else:
    try:
      #response.status_code,response.content,response.text
      headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
      my_response = requests.get(url_01,headers=headers)
      my_content = my_response.content.decode()
      my_html = etree.HTML(my_content)
      my_imgs = my_html.xpath('.//img/@src')
      print(my_imgs,"\n")
      for i in my_imgs:
        try:
          if(i.split('/')[-1].split('.')[-1].split('?')[0]=='png'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0]+".jpg")
            png_to_jpg(i.split('/')[-1].split('?')[0])
          elif(i.split('/')[-1].split('.')[-1].split('?')[0]=='jpg'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0])    
        except:
          pass
    except:
      pass  


  if(len(my_imgs_now)==0  or url_01==''):
    print('圖片網址設定失敗，使用預設圖片')
    url_01="https://64.media.tumblr.com/464a8a1770ac6b2c09c80232911673b7/b6713c87a0ba23de-b6/s1280x1920/0306a7f325e1c10e768a26ac2e1ea805ad3211d5.jpg"
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])  
  else:  
    print('圖片網址設定成功')

def get_image_size(my_img):
  im = Image.open(my_img)
  return im.size
      

In [ ]:
my_save_img()

remove_img=list(enumerate(my_imgs_now[0:]))
for i in remove_img:
    img_repeat=i[1]
    for ii in remove_img[i[0]+1:]:
        if(img_repeat==ii[1]):
            my_imgs_now.remove(img_repeat) 
            break
            
print(my_imgs_now)

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle)

In [ ]:
detector01=detector.signatures["default"]

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
def run_detector(detector, path): 
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(Image.fromarray(image_with_boxes))

In [ ]:
for i in my_imgs_now:
  print(i)
  run_detector(detector01,path+i)